{{title_s1_2}}

{{intro}}

In the last notebook, we demonstrated reading a larger-than-memory dataset using GDAL virtual datasets. However, reading the data in this way caused important metadata to be lost. This notebook details steps of parsing important metadata stored in filenames and separate files and formatting it so that it can be combined with the `Xr.Dataset` holding the physical observables, Sentinel-1 backscatter imagery for two different polarizations and a layover-shadow mask. 

::::{tab-set}
:::{tab-item} Outline  
(content.Section_A)=
**[A. Read and inspect initial metadata](#a-read-data-and-inspect-initial-metadata)**

- 1) Rename `band_data` variables to appropriate names  
- 2) What metadata currently exists?  

(content.Section_B)=
**[B. Add metadata from file name](#b-add-metadata-from-file-name)**  

- 1) Parse file name  
- 2) Extract and format acquisition dates   
- 3) Combine data cubes 

(content.Section_C)=
**[C. Time-varying metadata](#c-time-varying-metadata)** 

- 1) Apply `parse_fname_metadata()` to each element of each variable list, making a list of dictionaries   
- 2) Create tuple of metadata for each type of information  
- 3) Assign metadata tuple to a coordinate variable that exists along the time dimension of an Xarray dataset 

(content.Section-D)=
**[D. Add metadata from README file](#d-add-metadata-from-readme-file)**  

- 1) Extract granule ID  
- 2) Build granule ID coordinate array  

In [2]:
import cf_xarray
import geopandas as gpd
import markdown
import os
import pandas as pd
import pathlib
import re
import xarray as xr

import s1_tools

In [3]:
cwd = pathlib.Path.cwd()
tutorial2_dir = pathlib.Path(cwd).parent

## A. Read data and inspect initial metadata

Building the GDAL virtual dataset combines the stack of images by assigning each image to a `'band'` dimension. However, while we now have the data read into memory, we *don't* have important metadata that we need in order to interpret the data.

A good first step here is to check out the README file that is located within each scene directory. This has important metadata that we need as well as instructuions for how to interpret the file names. Check that out now if you haven't yet. 

Next, we have to figure out how to get the information from the README files and the file names and organize it so that it is most useful to us as we try to interpret the Sentinel-1 backscatter imagery. 

First, read the data using Xarray and the VRT files.

In [4]:
ds_vv = xr.open_dataset('../data/vrt_files/s1_stackVV.vrt', chunks='auto').squeeze()
ds_vh = xr.open_dataset('../data/vrt_files/s1_stackVH.vrt', chunks='auto')
ds_ls = xr.open_dataset('../data/vrt_files/s1_stackLS.vrt', chunks='auto')

### 1) Rename `band_data` variables to appropriate names

In [5]:
ds_vv = ds_vv.rename({'band_data':'vv'})
ds_vh = ds_vh.rename({'band_data':'vh'})
ds_ls = ds_ls.rename({'band_data':'ls'})

### 2) What metadata currently exists? 

Next, use `cf_xarray` to parse the metadata contained in the dataset.

In [ ]:
ds_vv.cf

As you can see, we're lacking a lot of information, but we do have spatial reference information that allows us to understand how the data cube's pixels correspond to locations on Earth's surface (we also know this from the README file).

In [ ]:
ds_vv

The `x` and `y` dimensions have associated coordinate variables and there is a `'spatial_ref'` variable that gives coordinate reference system information about those coordinates. In the attributes of `'spatial_ref'`, we see that our object is projected in local UTM (Universal Transverse Mercator) zone 45N. This tells us that the units of the coordinate arrays are meters. Read more about UTM zones [here](https://www.usgs.gov/faqs/what-does-term-utm-mean-utm-better-or-more-accurate-latitudelongitude).

In [ ]:
ds_vv['spatial_ref'].attrs

Great, we know where our data is located in space. But, with the data in its current form, we don't know much else about how or when it was collected. The rest of this notebook will demonstrate finding that information from both the file names of the individual files and the other files located in the scene directories. 

## B. Add metadata from file name 

Perhaps most importantly, we need to know when these observations were collected in time.

As we know from the README, information about acquisitions dates and more is stored in the directory name for each scene as well as the individual file names. 

In [ ]:
ds_vh.band

### 1) Parse file name
The function below extracts metadata from the filename of each GeoTIFF file in the time series. It uses [regex expressions](https://docs.python.org/3/howto/regex.html) to parse the different types of information stored within the file name based on the instructions from the README. Once parsed, the information is stored as a dictionary where each key is the type of information (ie. Acquisition start date and time, polarization, imaging mode, etc.) and each value is the corresponding information from the file name. 

In [10]:
def parse_fname_metadata(input_fname: str) -> dict:
    """Function to extract information from filename and separate into expected variables based on a defined schema."""
    # Define schema
    schema = {
        "sensor": (3, r"S1[A-B]"),  # schema for sensor
        "beam_mode": (2, r"[A-Z]{2}"),  # schema for beam mode
        "acq_date": (15, r"[0-9]{8}T[0-9]{6}"),  # schema for acquisition date
        "pol_orbit": (3, r"[A-Z]{3}"),  # schema for polarization + orbit type
        "terrain_correction_pixel_spacing": (
            5,
            r"RTC[0-9]{2}",
        ),  # schema for terrain correction pixel spacing
        "processing_software": (
            1,
            r"[A-Z]{1}",
        ),  # schema for processing software (G = Gamma)
        "output_info": (6, r"[a-z]{6}"),  # schema for output info
        "product_id": (4, r"[A-Z0-9]{4}"),  # schema for product id
        "prod_type": ((2, 6), (r"[A-Z]{2}", r"ls_map")),  # schema for polarization type
    }

    # Remove prefixs
    input_fname = input_fname.split("/")[-1]
    # Remove file extension if present
    input_fname = input_fname.removesuffix(".tif")
    # Split filename string into parts
    parts = input_fname.split("_")

    # l-s map objects have an extra '_' in the filename. Remove/combine parts so that it matches schema
    if parts[-1] == "map":
        parts = parts[:-1]
        parts[-1] = parts[-1] + "_map"

    # Check that number of parts matches expected schema
    if len(parts) != len(schema):
        raise ValueError(
            f"Input filename does not match schema of expected format: {parts}"
        )

    # Create dict to store parsed data
    parsed_data = {}

    # Iterate through parts and schema
    for part, (name, (length_options, pattern_options)) in zip(parts, schema.items()):
        # In the schema we defined, items have an int for length or a tuple (when there is more than one possible lenght)
        # Make the int lengths into tuples
        if isinstance(length_options, int):
            length_options = (length_options,)
        # Same as above for patterns
        if isinstance(pattern_options, str):
            pattern_options = (pattern_options,)

        # Check that each length of each part matches expected length from schema
        if len(part) not in length_options:
            raise ValueError(f"Part {part} does not have expected length {len(part)}")
        # Check that each part matches expected pattern from schema
        if not any(re.fullmatch(pattern, part) for pattern in pattern_options):
            raise ValueError(
                f"Part {part} does not match expected patterns {pattern_options}"
            )

        # Special handling of a part (pol orbit) that has 3 types of metadata
        if name == "pol_orbit":
            parsed_data.update(
                {
                    "polarization_type": part[:1],  # Single (S) or Dual (D) pol
                    "primary_polarization": part[1:2],  # Primary polarization (H or V)
                    "orbit_type": part[
                        -1
                    ],  # Precise (p), Restituted (r) or Original predicted (o)
                }
            )
        # Format string acquisition date as a datetime time stamp
        elif name == "acq_date":
            parsed_data[name] = pd.to_datetime(part, format="%Y%m%dT%H%M%S")
        # Expand multiple variables stored in output_info string part
        elif name == "output_info":
            output_info_keys = [
                "output_type",
                "output_unit",
                "unmasked_or_watermasked",
                "notfiltered_or_filtered",
                "area_or_clipped",
                "deadreckoning_or_demmatch",
            ]

            output_info_values = [part[0], part[1], part[2], part[3], part[4], part[-1]]

            parsed_data.update(dict(zip(output_info_keys, output_info_values)))

        else:
            parsed_data[name] = part

    #remove prod type from parsed data dict
    prod_type = parsed_data.pop('prod_type')
    return parsed_data

:::{note}
We want to call `parse_fname_metadata()` on every GeoTIFF file in the time series. This means that we need the lists of file paths for each variable stored as GeoTIFF files that were created in the last notebook. 

Before calling `parse_fname_metadata()`, we recreate the lists by calling the same functions used in the last notebook from `s1_tools`.
:::

In [11]:
# Path to directory holding downloaded data
s1_asf_data = pathlib.Path(cwd.parents[3], 'sentinel1_rtc/data/asf_rtcs')
# Make file path lists for vv, vh, ls
filepaths_vv, filepaths_vh, filepaths_ls, filepaths_rm = s1_tools.make_filename_lists(s1_asf_data)

### 2) Extract and format acquisition dates

From each list of dictionaries created above, look at only the `acq_date` key, value pair in order to create lists of acquisition dates for each GeoTIFF file in the stack. 

In [12]:
acq_dates_vh = [
    parse_fname_metadata(filepaths_vh[file])["acq_date"].strftime("%m/%d/%YT%H%M%S")
    for file in range(len(filepaths_vh))
]
acq_dates_vv = [
    parse_fname_metadata(filepaths_vv[file])["acq_date"].strftime("%m/%d/%YT%H%M%S")
    for file in range(len(filepaths_vv))
]
acq_dates_ls = [
    parse_fname_metadata(filepaths_ls[file])["acq_date"].strftime("%m/%d/%YT%H%M%S")
    for file in range(len(filepaths_ls))
]

#Make sure they are  equal
assert acq_dates_vh == acq_dates_vv == acq_dates_ls, (
    "Acquisition dates lists for VH, VV and L-S Map do not match"
)

Let's pause and return to the data we read at the very beginning and look at how it relates to the acquisition date metadata we just parsed.

In each data cube object (`ds_vv`, `ds_vh` and `ds_ls`) there is a `'band'` dimension that has a length of 103. We know that this corresponds to time because of how we created the VRT files, but we don't know anything else about the timing of the observations stored in the data cubes.

In [ ]:
ds_vv

We also just created three lists: `acq_dates_vv`, `acq_dates_vh`, and `acq_dates_ls`. Each element of these lists contains a datetime-like object:

In [ ]:
acq_dates_vv[0]

We can assign these lists of dates to the data cubes using the `xr.assign_coords()` method together with `pd.to_datetime()`. [`pd.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) takes a string (the list elements) and turns it into a ['time-aware' object](https://pandas.pydata.org/docs/user_guide/timeseries.html). [`xr.assign_coords()](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html) assigns an array to a given coordinate. 

In [15]:
ds_vv = ds_vv.assign_coords(
    {"band": pd.to_datetime(acq_dates_vv, format="%m/%d/%YT%H%M%S")}
).rename({'band':'acq_date'})
ds_vh = ds_vh.assign_coords(
    {"band": pd.to_datetime(acq_dates_vh, format="%m/%d/%YT%H%M%S")}
).rename({'band':'acq_date'})
ds_ls = ds_ls.assign_coords(
    {"band": pd.to_datetime(acq_dates_ls, format="%m/%d/%YT%H%M%S")}
).rename({'band':'acq_date'})

In this case, we're assigning the time-aware array produced by passing `acq_dates_vv` to `pd.to_datetime()` to the `'band'` coordinate of the raster data cube.

### 3) Combine data cubes

Up until this point, we've been working with three raster data cube objects, one for each variable (`vv`, `vh` and `ls`). We can combine them into one data cube with multiple variables using [`xr.combine_by_coords()`](https://docs.xarray.dev/en/stable/generated/xarray.combine_by_coords.html#xarray.combine_by_coords)

In [16]:
ds = xr.combine_by_coords([ds_vv, ds_vh, ds_ls])

As discussed in the first notebook, the scenes are not read in temporal order when the VRT objects were created, which is why the current data cube is not in temporal order. This is okay, because while the scenes were not in temporal order, they were read in the *same* order across all variables, which is why we could create and assign the time dimensions (`acq_date`) as we did and merge the individual objects into one data cube. 

Now, let's use [`xr.sortby()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.sortby.html) to arrange the data cube's time dimension in chronological order.

In [17]:
ds = ds.sortby('acq_date')

In [ ]:
ds

The data cube is much more usable than when we first read it into memory. We've combined the multiple objects into one, replaced the `'band'` dimension that contained an array of arbitrary integers with a `'acq_date'` dimension of [`'datetime64[ns]'`](https://numpy.org/doc/2.2/reference/arrays.datetime.html) objects and sorted the data cube in chronological order.

:::{admonition} Taking a look at chunking

```
{image} ../imgs/s1_chunks.png
:alt chunks_image
:align_center
```

Each variable in `ds` has a total shape of (103,13379,1742) and is 89.59 GB. It is chunked so that each chunk is (11,1536,1536) and 99 MB, with 1080 total chunks per variable. 

Depending on your use-case, you may want to adjust the chunking of the object. For example, if you are interested in analyzing variability along the temporal dimension, it might make sense to re-chunk the dataset so that operations along the that dimension are more easily parallelized. For more detail, see the chunking discussion in Tutorial 1.  
(TODO, move this to general intro?) and the [Parallel Computing with Dask](https://docs.xarray.dev/en/stable/user-guide/dask.html) section of the Xarray documentation. 

## C. Time-varying metadata

So far we've only used the acquisition dates extracted from the file names, but the function we wrote `parse_fname_metadata()` holds additional information about how the scenes were imaged and processed. 

Similarly to how we attached the acquisition date information to the raster data cubes, we'd also like to organize the imaging and processing metadata so that it is stored in relation to the raster imagery in a way that helps us interpret the raster imagery.

Remember that we have a dictionary of metadata attributes for each step of the time series and each variable (product type):

In [ ]:
parse_fname_metadata(filepaths_vv[0])

In [ ]:
parse_fname_metadata(filepaths_vh[0])

Rather than store this information as `attrs` of the data cube as a whole (`ds.attrs`) or individual data variables(`ds['vv'].attrs`), **this information should be stored as coordinate variables of the data cube**. This way, there is a coordinate name (the key of each dict item) and an array that holds the value at each time step. 

The next several steps demonstrate how to execute this.

### 1) Apply `parse_fname_metadata()` to each element of each variable list, making a list of dictionaries

In [21]:
meta_attrs_list_vv = [
    parse_fname_metadata(filepaths_vv[file]) for file in range(len(filepaths_vv))
]

meta_attrs_list_vh = [
    parse_fname_metadata(filepaths_vh[file]) for file in range(len(filepaths_vh))
]

meta_attrs_list_ls = [
    parse_fname_metadata(filepaths_ls[file]) for file in range(len(filepaths_ls))
]

In [ ]:
len(list(meta_attrs_list_vv[0].keys()))

### 2) Create tuple of metadata for each type of information

To assign coordinate variables to an Xarray dataset, the metadata needs to be re-organized a bit more. Currently, we have a list of 103 dictionaries with identical keys; each key will become a coordinate variable. To assign a coordinate variable to a `xr.Dataset`, Xarray expects a tuple with the following form: `(coordinate name, coordinate array)` where the array is all of the values for a single key from all dictionaries in the list.  

In short, we need to go from 103 dictionaries with 14 keys each to 14 tuples, each holding the key name and an array with length 103.

In [23]:
def stacked_meta_tuple(input_ls):
    """takes a list of dictionaries where each dict is metadata for a given time step,
    returns a tuple. input list is re-organized to a tuple where each [0] values is a metadata category (ie. sensor) and
    [1] is a time series of the categories values over time (ie. S1A, S1A, S1A ...)"""

    #Start wtih first dictionary in list
    meta_dict = input_ls[0]
    ticker = 0
    #Initialize empty lists to hold attr dictionaries and keys
    attrs_dicts, keys_ls = [], []

    #Iterate through keys in first dictionary
    for key in meta_dict:
        if key == "acq_date": #Skip, already added as a dimensional coordinate variable
            pass
        else:
            # Make a new dict where the key is the key and the 
            # value is a list of the associated values of that 
            # key for every element (time step) of the initial list
            key_dict = {
                f"{key}": [input_ls[ticker][key] for ticker in range(len(input_ls))]
            }
            ticker += 1
            attrs_dicts.append(key_dict)
            keys_ls.append(key)

    full_tuple = tuple(zip(keys_ls, attrs_dicts))

    return full_tuple

In [24]:
metadata_tuple = stacked_meta_tuple(meta_attrs_list_vv)

This can be a bit confusing. To help illustrate the above function, let's break down the output a bit more. 

`metadata_tuple` is a tuple of tuples. Each inner tuple holds a string and a list.

In [ ]:
print(f"({metadata_tuple[0][0]},{metadata_tuple[0][1]}")

TODO not sure how / why but every image in the time series is lsited as Sentinel-1 when I don't think this is possible with the repeat times? It's like that in the dir /file names as well as the README files so I don't think its anything I did processing wise...

In [ ]:
print(f"({metadata_tuple[1][0]},{metadata_tuple[1][1]}")

This tells us that the beam mode of the images is 'Interferometric Wide'.

### 3) Assign metadata tuple to a coordinate variable that exists along the time dimension of an Xarray dataset

In [27]:
def apply_meta_coords(input_xr, input_tuple):
    """takes an input xarray object and a tuple of metadata created from the above fn.
    returns xr object with metadata from tuple applied
    """

    out_xr = xr.Dataset(
        input_xr.data_vars,
        coords={
            "x": input_xr.x.data,
            "y": input_xr.y.data,
        },
    )
    # now apply metadata coords
    for element in range(len(input_tuple)):
        key = input_tuple[element][0]
        coord = list(input_tuple[element][1].values())[0]
        out_xr.coords[f"{key}"] = ("acq_date", coord)

    return out_xr

In [28]:
ds_w_metadata = apply_meta_coords(ds, metadata_tuple)

In [ ]:
ds_w_metadata

Great, from a stack of files that make up a raster time series, we figured out how to take information stored in a every file name and assign it as non-dimensional coordinate variables of the data cube representing the raster time series. 

While it's good that we did this because there is nothing *ensuring* that the metadata in the file names is identical across time steps, many of these attributes are identical across the time series. This means that we can handle them more simply as attributes rather than coordinate variables, though we will still keep some metadata as coordinates. 

First, verify which metadata attributes are identical throughout the time series:

In [30]:
#Make list of coords that are along time dimension
coords_along_time_dim = [coord for coord in ds_w_metadata._coord_names if 'acq_date' in ds_w_metadata[coord].dims]
dynamic_attrs = []
static_attr_dict = {}
for i in coords_along_time_dim:
    if i != 'acq_date':
        # find coordinate array variables that have more than
        # one unique element along time dimension
        if len(set(ds_w_metadata.coords[i].data.tolist())) > 1:
            dynamic_attrs.append(i)
        else:
            static_attr_dict[i] = ds_w_metadata.coords[i].data[0]


Drop the static coordinates from the data cube:

In [31]:
ds_w_metadata = ds_w_metadata.drop_vars(list(static_attr_dict.keys()))

And add them as attributes

In [32]:
ds_w_metadata.attrs = static_attr_dict

In [ ]:
ds_w_metadata

## D. Add metadata from README file

Until now, all of the attribute information that we've added to the data cube came from file names of the individual scenes that make up the data cube. One piece of information not included in the file name is the original granule ID of the scene published by ESA. This is information about the source image, produced by ESA, and used by ASF when processing a SLC image into a RTC image. This is located in the README file within each scene directory. 


Having the granule ID can be helpful because it tells you if the satellite was in an ascending or a descending cycle when the image was captured. For surface conditions with diurnal fluctuations such as snowmelt, the timing of the image acquisition may be important.

:::{note} 
This notebook works through the steps to format and organize metadata while explaining the details behind these operations. It is meant to be an educational demonstration. There are packages available such as [Sentinelsat](https://sentinelsat.readthedocs.io/en/stable/) that simplify metadata retrieval of Sentinel-1 imagery.
:::

### 1) Extact granule ID

We first need to extract the text containing a scene's granule ID from that scene's README file. This is similar to parsing the file and directory names but this time we're extracting text from a markdown file. To do this, we use the [python-markdown](https://python-markdown.github.io/) package. From looking at the file, we know where the source granule information is located; we need to isolate it from the rest of the text in the file. The source granule information always follows the same pattern, so we can use string methods to extract it.



In [34]:
def extract_granule_id(filepath):
    ''' takes a filepath to the readme associated with an S1 scene and returns the source granule id used to generate the RTC imagery''' 

    # Use markdown package to read text from README
    md = markdown.Markdown(extensions=['meta'])
    # Extract text from file
    data = pathlib.Path(filepath).read_text()
    #this text precedes granule ID in readme
    pre_gran_str = 'The source granule used to generate the products contained in this folder is:\n'
    split = data.split(pre_gran_str)
    #isolate the granule id
    gran_id = split[1][:67]
    
    return gran_id

Before moving on, take a look at a single source granule ID string:

In [ ]:
granule_ls = [
        extract_granule_id(filepaths_rm[element])
        for element in range(len(filepaths_rm))
    ]
granule_ls[0]

Below is information from the ESA's [SentiWiki](https://sentiwiki.copernicus.eu/web/) on [Sentinel-1 Product Naming Conventions](https://sentiwiki.copernicus.eu/web/s1-products#S1Products-SARNamingConventionS1-Products-SAR-Naming-Convention). In addition to the sensor and the beam mode, it tells us that the product type is a 'Single Look Complex' (SLC) image and is Level-1 processed data. The two date-time like sections indicate the start (14 February 2022, 12:13:53) and stop (14 February 2022, 12:14:20) date and time of the observation which are followed by the absolute orbit number (041909), a mission data take ID (04FD6F) and a unique product ID (42AF). 

```{image} ../imgs/esa_s1_naming.png
align: center
```


TODO
- probably save clipping until next notebook. write function in scripts that does all of this metadata wrangling under the hood for the next notebooks or save to disk i cant remember whcih is better. oh wait maybe just clip at hte end of this notebook and write to disk. 

### 2) Build granule ID coordinate array

This is another situation of time-varying metadata. Once we have the indiviudal granule IDs, we must format them so that they can be attached to an Xarray object.

In [36]:
def make_coord_data(readme_fpaths_ls):
    """takes a list of the filepaths to every read me, extracts the granule ID.
    From granule ID, extracts acquisition date and data take ID.
    Returns a tuple of lists of acquisition dates and data take IDs."""

    #Make a list of all granules in time series
    granule_ls = [
        extract_granule_id(readme_fpaths_ls[element])
        for element in range(len(readme_fpaths_ls))
    ]
    # Define a schema for aquisition date
    schema = {
        "mission_identifier": (3, r"S1[A-B]"),  # schema for sensor
        "mode_beam_identifier": (2, r"[A-Z]{2}"),  # schema for beam mode
        "esa_product_type": (3, r"[A-Z]{3}"),  # schema for ESA product type
        "proc_lvl_class_pol": (4, f"[A-Z0-9]{{4}}"),  
        "acq_start": (15, r"[0-9]{8}T[0-9]{6}"),  # schema for acquisition dat
        "acq_stop" : (15, r"[0-9]{8}T[0-9]{6}"),  # schema for acquisition dat
        "orbit_no" : (6, r"[0-9]{6}"),  # schema for orbit number
        "data_take_id": (6, "A-Z0-9{6}"),  # schema for data take id

    }
    
    #Extract relevant metadata from granule ID
    all_granules_parsed_data = []
    for granule in granule_ls:
        #need to account for double under score
        parts = [s for s in granule.split('_') if len(s) > 0]
        #parts = granule.split("_")
        single_granule_parsed_data = {}
        for part, (name, (length, pattern)) in zip(parts, schema.items()):
            if name == 'acq_start': 
                single_granule_parsed_data['acq_start'] = pd.to_datetime(part, format="%Y%m%dT%H%M%S")
            elif name == 'data_take_id':
                single_granule_parsed_data[name] = part
        all_granules_parsed_data.append(single_granule_parsed_data)

    acq_dates = [granule["acq_start"] for granule in all_granules_parsed_data]
    data_take_ids = [granule["data_take_id"] for granule in all_granules_parsed_data]
   
    return (acq_dates, data_take_ids)

In [37]:
acquisition_dates, data_take_ids_ls = make_coord_data(filepaths_rm)

In [38]:
def create_da(variable_ls:list, variable:str, 
              acq_date_ls:list, desc:str):

    da = xr.DataArray(
        data = variable_ls,
        dims=["acq_date"],
        coords = {"acq_date": acq_date_ls},
        name = variable,
        attrs= {"description": desc}
    )
    da = da.sortby(da.acq_date)
    return da


In [39]:
data_take_id_da = create_da(variable_ls = data_take_ids_ls, 
                       variable='data_take_id',
                       acq_date_ls=acquisition_dates,
                       desc='ESA Mission data take ID')

Add `xr.DataArray` of ESA data take IDs to raster data cube of Sentinel-1 scenes as a coordinate variable:

In [40]:
ds_w_metadata.coords['data_take_ID'] = data_take_id_da

In [ ]:
ds_w_metadata

### Handle nodata values

Another important difference between the `xr.open_mfdataset()` and VRT approaches is the handling of the nodata values. In the object built using `xr.open_mfdataset()`, nodata values are set to NaN while building the object from VRT files converts nodata values to zeros. This can cause a stream of problems later on so it is best to convert them to NaNs now. You can check the handling of nodata values in your dataset by using the `rio.nodata` accessor (shown below) which you can read about [here](https://corteva.github.io/rioxarray/stable/getting_started/nodata_management.html).

In [ ]:
ds_w_metadata.vv.isel(acq_date=0).rio.nodata

### 3. Clip by spatial area of interest

Use a vector data frame to subset the large raster data cube to a smaller area of interest. In this case, the vector dataframe used is called `pc_aoi` because it is the spatial footprint used when accessing another version of the Sentinel-1 RTC dataseet processed by Planetary Comptuter (TODO clean up wording/refs and insert link to pc notebook).

In [ ]:
pc_aoi = gpd.read_file(
    "https://github.com/e-marshall/sentinel1_rtc/raw/main/hma_rtc_aoi.geojson"
)